In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

# Calibration study

In [ ]:
from calibration import get_calibration_function_median
from plotting_tools import save_fig

XLIM = [3000, 4500]
YLIM = [3, 50]
SIZE = [5, 5]

for motors in [0, "all45000"]:
    flag = "no" if motors == 0 else ""

    fig, ax = plt.subplots()
    fig.set_size_inches(*SIZE)
    calib_function_median, freqs = get_calibration_function_median(
        "2021_07_08_stepper_fast", mic_type="audio_deck", snr=5, motors=motors, ax=ax
    )
    ax.set_title(f"stepper, with motors")
    ax.legend(loc='upper left')
    ax.set_yscale("log")
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    save_fig(fig, f"plots/experiments/calibration_stepper_{flag}motors.pdf")

In [ ]:
from calibration import get_calibration_function_moving

from matplotlib import cm
cmap = cm.get_cmap('inferno')

exp_name = "2021_07_27_manual"
results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")

for motors in [0, "all45000"]:
    flag = "no" if motors == 0 else ""
    
    fig_total, ax_total = plt.subplots()
    fig_total.set_size_inches(*SIZE)
    get_calibration_function_moving(exp_name, motors=motors, fit_one_gain=False, ax=ax_total)
    ax_total.set_yscale("log")
    ax_total.set_ylim(*YLIM)
    ax_total.set_xlim(*XLIM)
    save_fig(fig_total, f"plots/experiments/calibration_manual_{flag}motors.pdf")
    
    rows = results_df.loc[results_df.motors==motors]
    fig, axs = plt.subplots(1, 4)
    fig.set_size_inches(15, 5)
    for mic_i in range(4):
        for i_row, row in rows.iterrows():
            
            #if np.any(np.abs(row.stft[:, mic_i, :]) > 30):
            #    print(f'mic{mic_i}{row.appendix}: problematic')
            #else:
            #    print(f'mic{mic_i}{row.appendix}: ok')
                
            for i, (stft, freq) in enumerate(zip(row.stft, row.frequencies_matrix)):
                color = cmap(i/row.stft.shape[0])
                axs[mic_i].plot(freq[freq > 0], np.abs(stft[mic_i, freq>0]), color=color, alpha=0.2)
                
        #ax_total.plot(freq[freq > 0], np.abs(stft_average[mic_i, freq>0]), color=f"C{mic_i}")
        #ax_total.grid()
        #ax_total.legend()

## Hovering study

In [ ]:
def get_positions(row, chosen_sweeps):
    START_Y = 0, #row.positions[0, 1]
    START_X = row.positions[0, 0]
    #print("starting at", START_X, START_Y)
    if row.appendix in ["", "_30"]:
        START_Y = 0.3 # m 
    else:
        print("using 0.5")
        START_Y = 0.5 # m 
    positions_cm = np.c_[row.positions[chosen_sweeps, 0] - START_X,
                         START_Y - row.positions[chosen_sweeps, 1], 
                         row.positions[chosen_sweeps, 2]] * 1e2
    if np.any(positions_cm[:, 2] < 30):
        return None
    return positions_cm

### use stepper motor data

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG_STEPPER, WALL_DISTANCE_CM_STEPPER
exp_name = "2021_07_08_stepper_fast"
distance = 15
calib = "stepper"

azimuth_deg = WALL_ANGLE_DEG_STEPPER
d_corr = distance - WALL_DISTANCE_CM_STEPPER

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
row = results_df.loc[results_df.distance == d_corr, :].iloc[0]

label = f"stepper at {distance} calibration {calib}"
mics = range(4) #[0, 1]

stft_list = list(row.stft)
freqs_list = list(row.frequencies_matrix)
pos_list = np.array([[0, distance]] * len(stft_list))

start_i = 0 # for naming only

row

### use hovering data

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

exp_name = "2021_07_27_hover"; 
distance = 30
calib = "moving"

azimuth_deg = WALL_ANGLE_DEG

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
row = results_df.loc[results_df.appendix == f"_{distance}", :].iloc[0]

label = f"hover at {distance} calibration {calib}"
mics = [0, 1]

# choose three sweeps to plot
start_i = 3
n_sweeps = 3

#for start_i in np.arange(row.stft.shape[0]-n_sweeps)[::3]:
chosen_sweeps = range(start_i, start_i+n_sweeps)
stft_list = row.stft[chosen_sweeps]
freqs_list = row.frequencies_matrix[chosen_sweeps]
pos_list = get_positions(row, chosen_sweeps) 
dist_average = np.mean([p[1] for p in pos_list])

In [ ]:
if calib == "moving":
    print("using moving")
    calib_function_median, freqs = get_calibration_function_moving(
        "2021_07_27_manual", motors="all45000", fit_one_gain=False
    )
elif calib == "stepper":
    print("using stepper dataset")
    calib_function_median, freqs = get_calibration_function_median(
        "2021_07_08_stepper_fast",
        motors="all45000",
        mic_type="audio_deck",
        snr=5,
        fit_one_gain=False,
    )

In [ ]:
from estimators import DistanceEstimator
from inference import Inference, eps_normalize
from plotting_tools import save_fig
from matplotlib import cm

cmap = cm.get_cmap('inferno')
distance_range = [0.0, 80.0]

#for start_i in np.arange(row.stft.shape[0]-n_sweeps-1)[::3]:
    #chosen_sweeps = range(start_i, start_i+n_sweeps)
    #stft_list = row.stft[chosen_sweeps]
    #freqs_list = row.frequencies_matrix[chosen_sweeps]
    #pos_list = get_positions(row, chosen_sweeps) 
    #if pos_list is None: # invalid position
    #    continue
    #dist_average = np.mean([p[1] for p in pos_list])

inf_machine = Inference()
inf_machine.add_calibration_function(calib_function_median)
inf_machine.add_geometry(distance_range, azimuth_deg)

fig_slice, ax_slice = plt.subplots(1, len(mics))
fig_slice.set_size_inches(3*len(mics), 3)

fig_prob, ax_prob = plt.subplots(1, len(mics))
fig_prob.set_size_inches(3*len(mics), 3)

fig_pos, ax_pos = plt.subplots()
fig_pos.set_size_inches(3, 3)
fig_total, ax_total = plt.subplots()
fig_total.set_size_inches(3, 3)


#### Treat all stfts individually
inf_machine = Inference()
inf_machine.add_calibration_function(calib_function_median)
inf_machine.add_geometry(distance_range, azimuth_deg)

distance_estimator_all = DistanceEstimator()

dist_average = np.mean([p[1] for p in pos_list])
for i, (stft, freqs, pos) in enumerate(zip(stft_list, freqs_list, pos_list)):
    #color = cmap(i/len(stft_list))
    color = f"C{i}"

    ax_pos.scatter(pos[0], pos[1], s=20.0, marker='x', color=color)

    inf_machine.add_data(np.abs(stft), freqs)
    inf_machine.filter_out_freqs()

    distance_estimator = DistanceEstimator()

    for mic_i, mic_idx in enumerate(mics):
        amps = np.abs(stft[mic_idx, :])
        ax_slice[mic_i].plot(freqs[freqs > 0], 
                      amps[freqs > 0], 
                      color=color,ls="-")
        dist, proba, diff = inf_machine.do_inference("bayes", mic_idx)

        distance_estimator.add_distribution(diff*1e-2, proba, mic_idx)
        distance_estimator_all.add_distribution(diff*1e-2, proba, mic_idx)

        ax_prob[mic_i].plot(dist, proba, label=f'mic{mic_idx}', color=color)

        #ax_slice.plot(freqs[freqs>0], calib_function_median(freqs[freqs>0])[mic_i], 
        #            color='C0')
    dist, proba = distance_estimator.get_distance_distribution(
        method='sum',
        distances_m=1e-2*np.arange(*distance_range, step=1.0)
    )
    ax_total.plot(dist*1e2, eps_normalize(proba, 1e-3), color=color, ls='-', 
                  label=f'probability sweep {i}')
dist, proba = distance_estimator_all.get_distance_distribution(
    method='sum',
    distances_m=1e-2*np.arange(*distance_range, step=1.0),
    verbose=True
)
ax_total.plot(dist*1e2, eps_normalize(proba, 1e-3), color='k', ls='-', 
              label='averaged probabilities')


#### Create average of stfts. 
stft_array = np.array(stft_list)
stft = np.median(stft_array, axis=0)

inf_machine.add_data(np.abs(stft), freqs)

freqs = freqs_list[0]
distance_estimator = DistanceEstimator()
for mic_i, mic_idx in enumerate(mics):
    amps = np.abs(stft[mic_idx, :])
    ax_slice[mic_i].plot(freqs[freqs > 0], 
                         amps[freqs > 0], 
                         color='k',ls="--")
    dist, proba, diff = inf_machine.do_inference("bayes", mic_idx)
    distance_estimator.add_distribution(diff*1e-2, proba, mic_idx)
    ax_prob[mic_i].plot(dist, proba, label=f'mic{mic_idx}', color='k', ls='--')
    #ax_slice.plot(freqs[freqs>0], calib_function_median(freqs[freqs>0])[mic_i], 
    #            color='C0')
dist, proba = distance_estimator.get_distance_distribution(
    method='sum',
    distances_m=1e-2*np.arange(*distance_range, step=1.0)
)
ax_total.plot(dist*1e2, eps_normalize(proba, 1e-3), color='k', ls='--', 
              label='averaged sweeps')


#### Plot decorations
ax_prob[0].set_ylabel('probability')
ax_slice[0].set_ylabel('amplitude')
for m, mic_idx in enumerate(mics):
    ax_prob[m].set_xlabel('distance [cm]')
    ax_prob[m].set_title(f'mic{m}')
    ax_prob[m].grid()

    ax_slice[m].set_xlabel('frequency [Hz]')
    ax_slice[m].set_title(f'mic{m}')
    ax_slice[m].grid()

ax_pos.set_xlabel('x [cm]')
ax_pos.set_ylabel('y [cm]')
ax_pos.axis('equal')
ax_pos.set_title(label)
ax_pos.grid()

ax_total.set_xlabel('distance [cm]')
ax_total.set_ylabel('probability')
ax_total.set_title(label)
ax_total.axvline(dist_average, ls='-', color='C5', label='average distance')
ax_total.legend(loc='upper left', bbox_to_anchor=[1.0, 1.0])
ax_total.grid()
save_fig(fig_total, f'plots/experiments/tests_new/{exp_name}_{label.replace(" ","_")}_start{start_i}.pdf')